In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('733_label_data.csv')
print(len(df))

df

12113


,id,date,tweet,tidyTweet,Label,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1484309873059934208,2022-01-20 15:40:25,@77WABCradio @RudyGiuliani I suppose all the e...,"I suppose all the election fraud, the Biden ...",1.0,NaN,NaN,NaN
1,1484309738808840192,2022-01-20 15:39:53,The press had a lot of questions about Ukraine...,The press had a lot of questions about Ukraine...,0.0,NaN,NaN,NaN
2,1484309298867933186,2022-01-20 15:38:08,"Nevada, vote for the candidate who has the rig...","Nevada, vote for the candidate who has the rig...",0.0,NaN,NaN,NaN
3,1484307683532754944,2022-01-20 15:31:43,@POTUS PLEASE TAKE ACTION AND ABOLISH YOUR TRU...,PLEASE TAKE ACTION AND ABOLISH YOUR TRUTH IN S...,0.0,NaN,NaN,NaN
4,1484306314432032768,2022-01-20 15:26:17,#Trump Backs Banning #NancyPelosi from #StockT...,#Trump Backs Banning #NancyPelosi from #StockT...,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
12108,1502633293023428610,2022-03-12 05:11:09,Seems to be a new strain of Covid coming our w...,Seems to be a new strain of Covid coming our w...,0.0,NaN,NaN,NaN
12109,1502623553891741701,2022-03-12 04:32:27,@MusicManVA 🐸🐷 Missing❗️ 🌸 #WheresFauci??? ...,🐸🐷 Missing❗️ 🌸 #WheresFauci??? 😆,1.0,NaN,NaN,NaN
12110,1502621900618489856,2022-03-12 04:25:53,#WheresFauci ?? His stubby little fingerprin...,#WheresFauci ?? His stubby little fingerprin...,1.0,NaN,NaN,NaN
12111,1502607009652293634,2022-03-12 03:26:42,good morning...#WheresFauci,good morning...#WheresFauci,0.0,NaN,NaN,NaN


In [4]:
df = df[~df['tweet'].str.contains('#depop ')].reset_index(drop=True)
print(len(df))

df

11813


,id,date,tweet,tidyTweet,Label,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1484309873059934208,2022-01-20 15:40:25,@77WABCradio @RudyGiuliani I suppose all the e...,"I suppose all the election fraud, the Biden ...",1.0,NaN,NaN,NaN
1,1484309738808840192,2022-01-20 15:39:53,The press had a lot of questions about Ukraine...,The press had a lot of questions about Ukraine...,0.0,NaN,NaN,NaN
2,1484309298867933186,2022-01-20 15:38:08,"Nevada, vote for the candidate who has the rig...","Nevada, vote for the candidate who has the rig...",0.0,NaN,NaN,NaN
3,1484307683532754944,2022-01-20 15:31:43,@POTUS PLEASE TAKE ACTION AND ABOLISH YOUR TRU...,PLEASE TAKE ACTION AND ABOLISH YOUR TRUTH IN S...,0.0,NaN,NaN,NaN
4,1484306314432032768,2022-01-20 15:26:17,#Trump Backs Banning #NancyPelosi from #StockT...,#Trump Backs Banning #NancyPelosi from #StockT...,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11808,1502633293023428610,2022-03-12 05:11:09,Seems to be a new strain of Covid coming our w...,Seems to be a new strain of Covid coming our w...,0.0,NaN,NaN,NaN
11809,1502623553891741701,2022-03-12 04:32:27,@MusicManVA 🐸🐷 Missing❗️ 🌸 #WheresFauci??? ...,🐸🐷 Missing❗️ 🌸 #WheresFauci??? 😆,1.0,NaN,NaN,NaN
11810,1502621900618489856,2022-03-12 04:25:53,#WheresFauci ?? His stubby little fingerprin...,#WheresFauci ?? His stubby little fingerprin...,1.0,NaN,NaN,NaN
11811,1502607009652293634,2022-03-12 03:26:42,good morning...#WheresFauci,good morning...#WheresFauci,0.0,NaN,NaN,NaN


In [7]:
df = df[~df['tweet'].str.contains('#Depop ')].reset_index(drop=True)
print(len(df))

df

11794


,id,date,tweet,tidyTweet,Label,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1484309873059934208,2022-01-20 15:40:25,@77WABCradio @RudyGiuliani I suppose all the e...,"I suppose all the election fraud, the Biden ...",1.0,NaN,NaN,NaN
1,1484309738808840192,2022-01-20 15:39:53,The press had a lot of questions about Ukraine...,The press had a lot of questions about Ukraine...,0.0,NaN,NaN,NaN
2,1484309298867933186,2022-01-20 15:38:08,"Nevada, vote for the candidate who has the rig...","Nevada, vote for the candidate who has the rig...",0.0,NaN,NaN,NaN
3,1484307683532754944,2022-01-20 15:31:43,@POTUS PLEASE TAKE ACTION AND ABOLISH YOUR TRU...,PLEASE TAKE ACTION AND ABOLISH YOUR TRUTH IN S...,0.0,NaN,NaN,NaN
4,1484306314432032768,2022-01-20 15:26:17,#Trump Backs Banning #NancyPelosi from #StockT...,#Trump Backs Banning #NancyPelosi from #StockT...,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11789,1502633293023428610,2022-03-12 05:11:09,Seems to be a new strain of Covid coming our w...,Seems to be a new strain of Covid coming our w...,0.0,NaN,NaN,NaN
11790,1502623553891741701,2022-03-12 04:32:27,@MusicManVA 🐸🐷 Missing❗️ 🌸 #WheresFauci??? ...,🐸🐷 Missing❗️ 🌸 #WheresFauci??? 😆,1.0,NaN,NaN,NaN
11791,1502621900618489856,2022-03-12 04:25:53,#WheresFauci ?? His stubby little fingerprin...,#WheresFauci ?? His stubby little fingerprin...,1.0,NaN,NaN,NaN
11792,1502607009652293634,2022-03-12 03:26:42,good morning...#WheresFauci,good morning...#WheresFauci,0.0,NaN,NaN,NaN


In [8]:
df.to_csv('dataset.csv')